In [1]:
# install syft if we are on google colab
import sys, os; os.system("pip install --pre syft") if "google.colab" in sys.modules else ""
import syft as sy
print(f"You're running syft version: {sy.__version__}")

/home/shubham/anaconda3/envs/syft/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


You're running syft version: 0.7.0-beta.18


In [2]:
domains = {} # our logged in domain clients
domain_credentials = set() # our set of domain credentials

In [3]:
# enter the dict given from the data owner
def add_credentials(credentials_dict_list):
    for credentials_dict in credentials_dict_list:
        fs = frozenset(credentials_dict.items())
        domain_credentials.add(fs)

In [4]:
add_credentials(
    [
        {'url': 'localhost', 'name': 'Samantha Carter', 'email': 'info@openmined.org', 'password': 'changethis', 'dataset_name': 'MedNIST Data 1/10'},
    ]   
)

In [5]:
domain_credentials

{frozenset({('dataset_name', 'MedNIST Data 1/10'),
            ('email', 'info@openmined.org'),
            ('name', 'Samantha Carter'),
            ('password', 'changethis'),
            ('url', 'localhost')})}

In [6]:
def login_to_domains(force: bool = False):
    for fd in domain_credentials:
        credentials = dict(fd)
        if credentials["url"] not in domains or force:
            try:
                details = credentials.copy()
                del details["name"]
                del details["dataset_name"]
                client = sy.login(**details, port=8081)
                domains[credentials["url"]] = client
            except Exception as e:
                print(e)

In [7]:
login_to_domains()


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to localhost... done! 	 Logging into canada... done!


In [8]:
#domain_client = domains['20.237.223.58']

In [9]:
domain_client = list(domains.values())[0]

In [10]:
domain_client.datasets[0]

Dataset: MedNIST Data 1/10
Description: The MedNIST dataset was gathered from several sets from TCIA, the RSNA Bone Age Challenge, and the NIH Chest X-ray dataset. The dataset is kindly made available by Dr. Bradley J. Erickson M.D., Ph.D. (Department of Radiology, Mayo Clinic) under the Creative Commons CC BY-SA 4.0 license.
Label Count: 6
Label Mapping: {"AbdomenCT": 0, "BreastMRI": 1, "CXR": 2, "ChestCT": 3, "Hand": 4, "HeadCT": 5}
Image Dimensions: (64, 64)
Total Images: 5895




Asset Key,Type,Shape
"[""train_images""]",,"(4731, 4096)"
"[""train_labels""]",,"(4731,)"
"[""val_images""]",,"(626, 4096)"
"[""val_labels""]",,"(626,)"
"[""test_images""]",,"(538, 4096)"
"[""test_labels""]",,"(538,)"


In [11]:
data = domain_client.datasets[0]

X_train = data["train_images"]
Y_train = data["train_labels"]

X_dev = data["train_images"]
Y_dev = data["train_labels"]

In [12]:
(X_train, Y_train), (X_dev, Y_dev)

((<TensorPointer -> canada:2a3b1cb6f47d41728737d79f1182272f>,
  <TensorPointer -> canada:84d45abe84e9485ab97c2f5a7743b494>),
 (<TensorPointer -> canada:2a3b1cb6f47d41728737d79f1182272f>,
  <TensorPointer -> canada:84d45abe84e9485ab97c2f5a7743b494>))

In [13]:
!pip install sklearn

In [14]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import preprocessing
from tqdm import tqdm

In [15]:
# Preprocess the images

X_train = X_train.T
X_train = X_train / 255.0

X_dev = X_dev.T
X_dev = X_dev / 255.0

In [16]:
def init_params():
    W1 = np.random.rand(6, 4096) - 0.5
    b1 = np.random.rand(6, 1) - 0.5
    W2 = np.random.rand(6, 6) - 0.5
    b2 = np.random.rand(6, 1) - 0.5
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1@(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2@(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2@(A1.T)
    db2 = 1 / m * dZ2.sum()
    dZ1 = W2.T@(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1@(X.T)
    db1 = 1 / m * dZ1.sum()
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [17]:
def get_predictions(A2):
    return np.argmax(A2)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    print("[INFO]: Starting training!\n")
    for i in tqdm(range(iterations)):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 40 == 0:
            print("Predicition will be availabe in the next iteration.....Thank you for your support.")
            # predictions = get_predictions(A2)
            # print("Iteration: ", i, " Accuracy : " ,get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [19]:
X_train.public_shape = (4096, 4731)
X_train.public_dtype = np.int64()
Y_train.public_shape = (4731,)
Y_train.public_shape = np.int64()

In [31]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 1)

[INFO]: Starting training!



  0%|                                                                                                                                            | 0/1 [00:00<?, ?it/s][2022-05-12T17:35:52.252392+0530][CRITICAL][logger]][106935] UnknownPrivateException has been triggered.
[2022-05-12T17:35:52.253280+0530][CRITICAL][logger]][106935] Request to access data length rejected.
  0%|                                                                                                                                            | 0/1 [00:00<?, ?it/s]

{'request_block': True, 'timeout_secs': 25, 'delete_obj': False}


ValueError: matmul: Input operand 1 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)